In [8]:
import simpy
import numpy as np
import functools
import matplotlib.pyplot as plt
from Stats import Entity, Resource, Source

## Description of problem: 

You have to model a restaurant. Arrivals come in poisson distribution

They must order. The Kitchen is described as follows: 
 - 2 cooks
 - 3 things to order:
   - fish -- 15 mins to prepare
   - burger -- 5 mins to prepare
   - salad -- 8 mins to prepare
   
Then must wait for a seat
- people will sit for between 20-30 minutes. 
- Capacity: 10
- Assume all attendees are dining alone (no need to worry about specific party capacity).


In [9]:
# First, set up interarrival generators and classes to represent resources
# for all of your problem parameters

# To use simpy is to understand generators
class Kitchen(Resource):
    def __init__(self, env, *args, **kwargs):
        self.name = "kitchen"
        self.env = env
        super().__init__(env, *args, **kwargs)
    
    def next_service_time(self, entity):
        """
        Defines preparation time for each dish
        """
        if entity.attributes["food"] == "Fish":
            return 15
        elif entity.attributes["food"] == "Burger":
            return 5
        elif entity.attributes["food"] == "Salad":
            return 8
        else:
            raise Exception("Restaurant does not serve this kind of food")
        
 
class DiningRoom(Resource):
    def __init__(self, env, *args, **kwargs):
        self.name = "DiningRoom"
        self.env = env
        super().__init__(env, *args, **kwargs)
    
    def next_service_time(self, entity):
        """
        logic for calculating next service time for resource
        """
        return np.random.randint(20, 30) # 20 to 30 minute seating per entity

class CustomerSource(Source):
    def __init__(self, env, *args, **kwargs):
        self.count = 0
        self.foods = ["Fish", "Burger", "Salad"]
        super().__init__(env, *args, **kwargs)
    
    def interarrival_time_generator(self):
        for i in [5, 5, 10, 10, 30, 5, 1]:
            yield i
    
    def sample_from_foods(self):
        return np.random.choice(self.foods, p=[0.1,0.7,0.2])
    
    def build_entity(self, creation_time):
        self.count += 1
        food = self.sample_from_foods()
        entity = Customer(env, f"Customer {self.count}", creation_time, food=food)
        return entity

# entities are in charge of the order in which interactions with resources occur
class Customer(Entity):
    def __init__(self, env, name, creation_time, *args, **kwargs):
        self.env = env
        super().__init__(env, name, creation_time, *args, **kwargs)
    
    # template method for all of the resources it needs to interact with
    def process(self, kitchen, dining_room):
        # interface with kitchen
        request = self.request_resource(kitchen)
        print(f'{self.name} requesting Chef: {self.env.now}')
        yield request
        
        print(f'{self.name} started Cooking: {self.env.now}')        
        self.start_service_at_resource(kitchen)
        cook_time = kitchen.next_service_time(self)
        print(f"{self.name} cook time: {cook_time} for food {self.attributes['food']}")
        yield self.env.timeout(cook_time)
        
        print(f'{self.name} finished at kitchen: {self.env.now}')
        self.release_resource(kitchen, request)
        
        # interface with dining_room
        request = self.request_resource(dining_room)
        print(f'{self.name} requesting Table: {self.env.now}')
        yield request
        
        print(f'{self.name} started Seating At Table: {self.env.now}')        
        self.start_service_at_resource(dining_room)
        seat_time = dining_room.next_service_time(self)
        print(f'{self.name} seat time is: {seat_time}')
        yield self.env.timeout(seat_time)
        
        print(f'{self.name} finished in dining room: {self.env.now}')
        self.release_resource(dining_room, request)

        return self

In [4]:
def process(env, customer_source, kitchen, dining_room):
    """ Generate arrivals into our simulation
    
    Arguments:   
     env:  Our simulation environment
     source: the source of entities for our simulation
     resource: Shared resource
    """
    
    def dispose(entity):
        return lambda _: entity.dispose()
    for arrival_time, customer in customer_source.next_entity():
        yield arrival_time # wait for the next entity to appear
        print(f"new customer arriving {customer.name}")
        processed_entity = env.process(customer.process(kitchen, dining_room))
        processed_entity.callbacks.append(dispose(customer))
        print("finished appending callback")

In [5]:
# Setup and start the simulation
print('Restaurant Simulation\n')
env = simpy.Environment()

source = CustomerSource(env)
dining_room = DiningRoom(env,  capacity=2)
kitchen = Kitchen(env, capacity=2)

Restaurant Simulation



In [6]:
env.process(process(env, source, kitchen, dining_room))
env.run()

new customer arriving Customer 1
finished appending callback
Customer 1 requesting Chef: 5
> <ipython-input-3-17ee12d78336>(69)process()
-> yield request


(Pdb)  c


Customer 1 started Cooking: 5
Customer 1 cook time: 8 for food Salad
new customer arriving Customer 2
finished appending callback
Customer 2 requesting Chef: 10
> <ipython-input-3-17ee12d78336>(69)process()
-> yield request


(Pdb)  c


Customer 2 started Cooking: 10
Customer 2 cook time: 5 for food Burger
Customer 1 finished at kitchen: 13
Customer 1 requesting Table: 13
Customer 1 started Seating At Table: 13
Customer 1 seat time is: 22
Customer 2 finished at kitchen: 15
Customer 2 requesting Table: 15
Customer 2 started Seating At Table: 15
Customer 2 seat time is: 27
new customer arriving Customer 3
finished appending callback
Customer 3 requesting Chef: 20
> <ipython-input-3-17ee12d78336>(69)process()
-> yield request


...

Customer 5 started Cooking: 60
Customer 5 cook time: 5 for food Burger
Customer 3 finished in dining room: 61
Customer 3 disposed: 61
Customer 4 finished in dining room: 62
Customer 4 disposed: 62
new customer arriving Customer 6
finished appending callback
Customer 6 requesting Chef: 65
> <ipython-input-3-17ee12d78336>(69)process()
-> yield request


(Pdb)  c


Customer 5 finished at kitchen: 65
Customer 5 requesting Table: 65
Customer 6 started Cooking: 65
Customer 6 cook time: 5 for food Burger
Customer 5 started Seating At Table: 65
Customer 5 seat time is: 20
new customer arriving Customer 7
finished appending callback
Customer 7 requesting Chef: 66
> <ipython-input-3-17ee12d78336>(69)process()
-> yield request


(Pdb)  c


Customer 7 started Cooking: 66
Customer 7 cook time: 15 for food Fish
Customer 6 finished at kitchen: 70
Customer 6 requesting Table: 70
Customer 6 started Seating At Table: 70
Customer 6 seat time is: 23
Customer 7 finished at kitchen: 81
Customer 7 requesting Table: 81
Customer 5 finished in dining room: 85
Customer 5 disposed: 85
Customer 7 started Seating At Table: 85
Customer 7 seat time is: 28
Customer 6 finished in dining room: 93
Customer 6 disposed: 93
Customer 7 finished in dining room: 113
Customer 7 disposed: 113


In [38]:
class Thing:
    
    def __init__(self):
        self.max = 10
    
    def __iter__(self):
        self.n = 0 
        return self
    
    def __next__(self):
        if self.n < self.max:
            self.n += 1
            return self.n
        else:
            raise StopIteration

for i in Thing():
    print(i)


1
2
3
4
5
6
7
8
9
10
